In [ ]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-719rpt_l
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-719rpt_l
     |████████████████████████████████| 174kB 6.4MB/s 
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-5lsl8ovz/pyfolio
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-5lsl8ovz/pyfolio
     |████████████████████████████████| 6.3MB 41.2MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
  Created wheel for finrl: filename=finrl-0.3.0-cp37-none-any.whl size=39054 sha256=2a4d7d534ab9aceba1d5dc07f9c225448a9be8111022cc6948edb29914301bed
  Stored in directory: /tmp/pip-ephem-wheel-cache-uyuusu0m/wheels/9c/19/bf/c644def96612df1ad42c94d5304966797eaa3221dffc5efe0b
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=2700013442d40611440a3231bb9d9b14b0b52b2cf899c13f3f6db40edf206cee
 

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt 
import datetime 
%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_baseline, get_daily_return
from pprint import pprint

In [ ]:
import sys 
sys.path.append("../FinRL-Library")
import itertools

In [ ]:
import os 
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
   os.makedirs("./" + config.RESULTS_DIR)

In [ ]:
df = YahooDownloader(start_date='2009-01-01', end_date='2021-01-01', ticker_list=config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

In [ ]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.787006,746015200,AAPL,4
1,2009-01-02,18.570000,19.520000,18.400000,15.657365,10955700,AXP,4
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
3,2009-01-02,44.910000,46.980000,44.709999,32.830360,7117200,CAT,4
4,2009-01-02,16.410000,17.000000,16.250000,12.505757,40980600,CSCO,4


In [ ]:
type(df['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
datasets = 'https://raw.githubusercontent.com/mariko-sawada/FinRL_with_fundamental_data/main/dow_30_fundamental_wrds.csv'
fund = pd.read_csv(datasets)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
fund.head()

In [ ]:
items = [
    'datadate', 
    'tic',
    'oiadpq', 
    'revtq',
    'niq', 
    'atq',  
    'teqq', 
    'epspiy', 
    'ceqq', 
    'cshoq',
    'dvpspq', 
    'actq',
    'lctq', 
    'cheq', 
    'rectq',  
    'cogsq',
    'invtq', 
    'apq',
    'dlttq', 
    'dlcq',
    'ltq' 
]

In [ ]:
fund_data= fund[items]

In [ ]:
# rename
fund_data = fund_data.rename(columns={
    'datadate':'date', # ngày 
    'oiadpq':'op_inc_q', # thu nhập từ hoạt động hàng quý 
    'revtq':'rev_q', # doanh thu hàng quý 
    'niq':'net_inc_q', # thu nhập ròng <=> lợi nhuận sau thuế
    'atq':'tot_assets', # tổng tài sản 
    'teqq':'sh_equity', # vốn do cổ đông sở hữu 
    'epspiy':'eps_incl_ex', # những mục ngoài 
    'ceqq':'com_eq', # vốn sở hữu chung giữa các cổ đông
    'cshoq':'sh_outstanding', # số cổ phiếu đang lưu hành 
    'dvpspq':'div_per_sh', # cổ tức trên một cổ phiếu 
    'actq':'cur_assets', # tổng tài sản hiện tại 
    'lctq':'cur_liabilities', # nợ ngắn hạn 
    'cheq':'cash_eq', # tiền mặt hiện có ($)
    'rectq':'receivables', # các khoản thu 
    'cogsq':'cogs_q', # giá vốn bán hàng
    'invtq':'inventories', # số lượng tồn 
    'apq': 'payables',# các khoản phải trả 
    'dlttq':'long_debt', # nợ dài hạn
    'dlcq':'short_debt', # nợ phải trả hiện tại
    'ltq':'tot_liabilities' # tổng nợ 
})

In [ ]:
fund_data.head()

,date,tic,op_inc_q,rev_q,net_inc_q,tot_assets,sh_equity,eps_incl_ex,com_eq,sh_outstanding,div_per_sh,cur_assets,cur_liabilities,cash_eq,receivables,cogs_q,inventories,payables,long_debt,short_debt,tot_liabilities
0,19990630,AXP,896.0,5564.0,646.0,132452.0,9762.0,2.73,9762.0,449.0,0.225,NaN,NaN,6096.0,46774.0,4668.0,448.0,22282.0,7005.0,24785.0,122690.0
1,19990930,AXP,906.0,5584.0,648.0,132616.0,9744.0,4.18,9744.0,447.6,0.225,NaN,NaN,5102.0,48827.0,4678.0,284.0,23587.0,6720.0,24683.0,122872.0
2,19991231,AXP,845.0,6009.0,606.0,148517.0,10095.0,5.54,10095.0,446.9,0.225,NaN,NaN,10391.0,54033.0,5164.0,277.0,25719.0,4685.0,32437.0,138422.0
3,20000331,AXP,920.0,6021.0,656.0,150662.0,10253.0,1.48,10253.0,444.7,0.225,NaN,NaN,7425.0,53663.0,5101.0,315.0,26379.0,5670.0,29342.0,140409.0
4,20000630,AXP,1046.0,6370.0,740.0,148553.0,10509.0,1.05,10509.0,1333.0,0.080,NaN,NaN,6841.0,54286.0,5324.0,261.0,29536.0,5336.0,26170.0,138044.0


Thực hiện việc tính các tỉ lệ tài chính bảo gồm: 

* Tỷ lệ OPM = Tổng thu nhập hoạt động hàng quý / Tổng doanh thu hằng quý 
* Tỷ lệ NPM = Tổng thu nhập ròng hằng quý / Tổng doanh thu hằng quý 
* Tỷ lệ ROA = Tổng thu nhập ròng hằng quý / Tổng tài sản 
* Tỷ lệ ROE = Tổng thu nhập ròng hằng quý / Vốn sở hữu của cổ đông
* EPS = Những mục bất thường 
* Tỷ lệ BPS = Vốn chủ sở hữu chung / Số cổ phiếu đang lưu hành 
* Tỷ lệ DPS = Số cổ tức trên một cổ phiếu 
* Tỉ lệ hiện tại (cur_ratio) = Tổng tài sản hiện tại / Tổng nợ phải trả hiện tại  
* Tỷ lệ thanh toán nhanh (quick_ratio) = (Tổng tiền mặt + Các khoản thu) / Tổng nợ phải trả 
* Tỷ lệ tiền mặt (cash_ratio) = Tổng tiền mặt / Tổng nợ phải trả 
* inv_turnover = Tổng bán ra / Tổng hàng tồn 
* acc_rec_turnover = Tổng doanh thu hằng quý / Các khoản thu 
* acc_pay_turnover = Tổng bán ra / Các khoản phải trả 
* Tỷ lệ nợ (debt_ratio) = Tổng nợ / Tổng tài sản 
* Tỷ lệ nợ vốn chủ sở hữu = Tổng nợ / Tổng vốn sở hữu của cổ đông 


In [ ]:
date = pd.to_datetime(fund_data['date'],format='%Y%m%d')

tic = fund_data['tic'].to_frame('tic')

# Tỷ lệ OPM = Tổng thu nhập hoạt động hàng quý / Tổng doanh thu hằng quý 
OPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='OPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        OPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        OPM.iloc[i] = np.nan
    else:
        OPM.iloc[i] = np.sum(fund_data['op_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])


# Tỷ lệ NPM = Tổng thu nhập ròng hằng quý / Tổng doanh thu hằng quý  
NPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='NPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        NPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        NPM.iloc[i] = np.nan
    else:
        NPM.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])

# Tỷ lệ ROA = Tổng thu nhập ròng hằng quý / Tổng tài sản 
ROA = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROA')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROA[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROA.iloc[i] = np.nan
    else:
        ROA.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['tot_assets'].iloc[i]

# Tỷ lệ ROE = Tổng thu nhập ròng hằng quý / Vốn sở hữu của cổ đông
ROE = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROE')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROE[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROE.iloc[i] = np.nan
    else:
        ROE.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['sh_equity'].iloc[i]        
        
EPS = fund_data['eps_incl_ex'].to_frame('EPS')     # Những mục bất thường 
BPS = (fund_data['com_eq']/fund_data['sh_outstanding']).to_frame('BPS')  # Tỷ lệ BPS = Vốn chủ sở hữu chung / Số cổ phiếu đang lưu hành 
DPS = fund_data['div_per_sh'].to_frame('DPS')   # Tỷ lệ DPS = Số cổ tức trên một cổ phiếu 
cur_ratio = (fund_data['cur_assets']/fund_data['cur_liabilities']).to_frame('cur_ratio')  # Tỉ lệ hiện tại (cur_ratio) = Tổng tài sản hiện tại / Tổng nợ phải trả hiện tại  
quick_ratio = ((fund_data['cash_eq'] + fund_data['receivables'] )/fund_data['cur_liabilities']).to_frame('quick_ratio')
# Tỷ lệ thanh toán nhanh (quick_ratio) = (Tổng tiền mặt + Các khoản thu) / Tổng nợ phải trả 
cash_ratio = (fund_data['cash_eq']/fund_data['cur_liabilities']).to_frame('cash_ratio')
# Tỷ lệ tiền mặt (cash_ratio) = Tổng tiền mặt / Tổng nợ phải trả 

inv_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='inv_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        inv_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        inv_turnover.iloc[i] = np.nan
    else:
        inv_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['inventories'].iloc[i]
# inv_turnover = Tổng bán ra / Tổng hàng tồn 
        
acc_rec_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_rec_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_rec_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_rec_turnover.iloc[i] = np.nan
    else:
        acc_rec_turnover.iloc[i] = np.sum(fund_data['rev_q'].iloc[i-3:i])/fund_data['receivables'].iloc[i]
# acc_rec_turnover = Tổng doanh thu hằng quý / Các khoản thu 

acc_pay_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_pay_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_pay_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_pay_turnover.iloc[i] = np.nan
    else:
        acc_pay_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['payables'].iloc[i]
# acc_pay_turnover = Tổng bán ra / Các khoản phải trả 
        

debt_ratio = (fund_data['tot_liabilities']/fund_data['tot_assets']).to_frame('debt_ratio')   # Tỷ lệ nợ (debt_ratio) = Tổng nợ / Tổng tài sản 
debt_to_equity = (fund_data['tot_liabilities']/fund_data['sh_equity']).to_frame('debt_to_equity') # Tỷ lệ nợ vốn chủ sở hữu = Tổng nợ / Tổng vốn sở hữu của cổ đông 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# tạo dataframe để chứa tất cả tỉ lệ 
ratios = pd.concat([date, tic, OPM, NPM, ROA, ROE, EPS, BPS, DPS, cur_ratio, quick_ratio, cash_ratio, 
                   inv_turnover, acc_rec_turnover, acc_pay_turnover, debt_ratio, debt_to_equity], axis =1)

In [ ]:
ratios

,date,tic,OPM,NPM,ROA,ROE,EPS,BPS,DPS,cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity
0,1999-06-30,AXP,NaN,NaN,NaN,NaN,2.73,21.741648,0.225,NaN,NaN,NaN,NaN,NaN,NaN,0.926298,12.568121
1,1999-09-30,AXP,NaN,NaN,NaN,NaN,4.18,21.769437,0.225,NaN,NaN,NaN,NaN,NaN,NaN,0.926525,12.610016
2,1999-12-31,AXP,NaN,NaN,NaN,NaN,5.54,22.588946,0.225,NaN,NaN,NaN,NaN,NaN,NaN,0.932028,13.711937
3,2000-03-31,AXP,0.154281,0.110742,0.012611,0.185312,1.48,23.055993,0.225,NaN,NaN,NaN,46.0635,0.319717,0.550059,0.931947,13.694431
4,2000-06-30,AXP,0.151641,0.108436,0.0128574,0.181749,1.05,7.883721,0.080,NaN,NaN,NaN,57.2529,0.324467,0.505925,0.929258,13.135788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,2020-03-31,V,0.667517,0.521213,0.129058,0.271736,2.85,13.647142,0.300,1.248714,1.140070,0.955150,inf,6.11635,2.69754,0.525062,1.105537
2452,2020-06-30,V,0.668385,0.519867,0.120448,0.264075,3.92,14.203947,0.300,1.553478,1.443292,1.221925,inf,5.06313,1.88951,0.543886,1.192433
2453,2020-09-30,V,0.654464,0.52129,0.107873,0.241066,4.90,14.653484,0.300,1.905238,1.784838,1.579807,inf,5.62857,2.73037,0.552515,1.234714
2454,2020-12-31,V,0.638994,0.480876,0.0944222,0.201545,1.42,15.908283,0.320,2.121065,1.969814,1.700081,inf,4.72531,2.34787,0.531507,1.134505


In [ ]:
# đổi giá trị NaN và inf thành 0
final_ratios = ratios.copy()
final_ratios = final_ratios.fillna(0)
final_ratios = final_ratios.replace(np.inf,0)

In [ ]:
final_ratios.head()

,date,tic,OPM,NPM,ROA,ROE,EPS,BPS,DPS,cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity
0,1999-06-30,AXP,0.000000,0.000000,0.000000,0.000000,2.73,21.741648,0.225,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.926298,12.568121
1,1999-09-30,AXP,0.000000,0.000000,0.000000,0.000000,4.18,21.769437,0.225,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.926525,12.610016
2,1999-12-31,AXP,0.000000,0.000000,0.000000,0.000000,5.54,22.588946,0.225,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.932028,13.711937
3,2000-03-31,AXP,0.154281,0.110742,0.012611,0.185312,1.48,23.055993,0.225,0.0,0.0,0.0,46.063492,0.319717,0.550059,0.931947,13.694431
4,2000-06-30,AXP,0.151641,0.108436,0.012857,0.181749,1.05,7.883721,0.080,0.0,0.0,0.0,57.252874,0.324467,0.505925,0.929258,13.135788


In [ ]:
# thêm các tỷ lệ tài chính vào dữ liệu giá 
list_ticker = df["tic"].unique().tolist()
list_date = list(pd.date_range(df['date'].min(),df['date'].max()))
combination = list(itertools.product(list_date,list_ticker))

# Merge
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(df,on=["date","tic"],how="left")
processed_full = processed_full.merge(final_ratios,how='left',on=['date','tic'])
processed_full = processed_full.sort_values(['tic','date'])


processed_full = processed_full.bfill(axis='rows')

In [ ]:
# Tính tỷ số P/E : là tỷ số đánh giá mối liên hệ giữa giá cổ phiếu hiện tại và tỷ số thu nhập trên cổ phần, 
                  # tỷ số P/E cho biết một nhà đầu tư sẵn sàng trả giá bao nhiêu cho một cổ phiếu trên sàn chứng khoán đó 
# Tính tỷ số P/B : là tỷ số để so sánh giá một cổ phiếu với giá trị ghi sổ của cố phiếu đó.
                  # tỷ số P/B giúp nhà đầu tư tìm kiếm các cổ phiếu có giá thấp mà thị trường bỏ qua 
# Tính lợi nhuận từ cổ tức
processed_full['PE'] = processed_full['close']/processed_full['EPS']
processed_full['PB'] = processed_full['close']/processed_full['BPS']
processed_full['Div_yield'] = processed_full['DPS']/processed_full['close']
processed_full = processed_full.drop(columns=['day','EPS','BPS','DPS'])

In [ ]:
processed_full.sort_values(['date','tic'],ignore_index=True)

,date,tic,open,high,low,close,volume,OPM,NPM,ROA,ROE,cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,PE,PB,Div_yield
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.787006,746015200.0,0.217886,0.163846,0.103222,0.183579,2.461857,2.039779,1.818995,54.403846,8.972003,4.269115,0.437727,0.778495,0.640691,0.102249,0.000000
1,2009-01-02,AXP,18.570000,19.520000,18.400000,15.657365,10955700.0,0.093973,0.072040,0.014094,0.108238,0.000000,0.000000,0.000000,0.000000,0.351354,0.653355,0.869784,6.679531,50.507629,1.450032,0.011496
2,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,0.047307,0.032525,0.026400,-2.870334,0.927883,0.368463,0.148507,2.329670,6.815203,2.076967,1.009198,-109.722986,39.012760,-35.751054,0.012374
3,2009-01-02,CAT,44.910000,46.980000,44.709999,32.830360,7117200.0,0.124545,0.066662,0.040891,0.415878,1.343293,0.890488,0.163158,3.540791,2.460351,8.472455,0.893715,9.089489,-172.791371,3.168809,0.012793
4,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.505757,40980600.0,0.234698,0.196418,0.097593,0.162793,2.792929,2.498162,2.170759,9.054201,6.844634,16.036800,0.400215,0.667591,19.850408,1.986886,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131455,2020-12-31,V,218.399994,219.820007,216.199997,218.074188,5922200.0,0.638994,0.480876,0.094422,0.201545,2.121065,1.969814,1.700081,0.000000,4.725314,2.347866,0.531507,1.134505,153.573372,13.708217,0.001467
131456,2020-12-31,VZ,58.060001,58.799999,58.020000,56.870125,12906300.0,0.230962,0.141165,0.041750,0.190741,1.376551,1.210691,0.589158,20.255568,3.797160,5.456577,0.781118,3.568671,13.225610,3.468883,0.011034
131457,2020-12-31,WBA,39.330002,40.000000,39.029999,39.166107,7696000.0,0.055546,0.032500,0.058790,0.000000,0.883715,0.201841,0.131341,6.289502,75.704738,7.523900,0.587123,0.000000,11.519443,2.354706,0.006064
131458,2020-12-31,WMT,144.199997,144.270004,142.850006,142.989090,5938000.0,0.153981,0.107187,0.154298,0.308230,1.309322,0.971255,0.507827,19.541401,13.960422,6.460123,0.499404,0.997621,155.422924,6.520801,0.002797


In [ ]:
# Thiết lập môi trường giao dịch 
# Thực hiện mô phỏng giao dịch từ 01/01/2019 - 31/12/2020
train = data_split(processed_full, '2009-01-01', '2019-01-01')
trade = data_split(processed_full, '2019-01-01', '2021-01-01')
print(len(train))
print(len(trade))

109530
21930


In [ ]:
train.head()

,date,tic,open,high,low,close,volume,OPM,NPM,ROA,ROE,cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,PE,PB,Div_yield
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.787006,746015200.0,0.217886,0.163846,0.103222,0.183579,2.461857,2.039779,1.818995,54.403846,8.972003,4.269115,0.437727,0.778495,0.640691,0.102249,0.000000
0,2009-01-02,AXP,18.570000,19.520000,18.400000,15.657365,10955700.0,0.093973,0.072040,0.014094,0.108238,0.000000,0.000000,0.000000,0.000000,0.351354,0.653355,0.869784,6.679531,50.507629,1.450032,0.011496
0,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,0.047307,0.032525,0.026400,-2.870334,0.927883,0.368463,0.148507,2.329670,6.815203,2.076967,1.009198,-109.722986,39.012760,-35.751054,0.012374
0,2009-01-02,CAT,44.910000,46.980000,44.709999,32.830360,7117200.0,0.124545,0.066662,0.040891,0.415878,1.343293,0.890488,0.163158,3.540791,2.460351,8.472455,0.893715,9.089489,-172.791371,3.168809,0.012793
0,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.505757,40980600.0,0.234698,0.196418,0.097593,0.162793,2.792929,2.498162,2.170759,9.054201,6.844634,16.036800,0.400215,0.667591,19.850408,1.986886,0.000000


In [ ]:
trade.head()

,date,tic,open,high,low,close,volume,OPM,NPM,ROA,ROE,cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,PE,PB,Div_yield
0,2019-01-01,AAPL,38.722500,39.712502,38.557499,38.439735,148158800.0,0.258891,0.227773,0.133360,0.430843,1.315382,1.134347,0.854114,23.571867,7.620024,3.781658,0.690466,2.230663,5.737274,1.672991,0.018991
0,2019-01-01,AXP,93.910004,96.269997,93.769997,91.803406,4175400.0,0.203479,0.160494,0.026811,0.237960,0.000000,0.000000,0.000000,0.000000,0.231669,0.279424,0.887329,7.875371,50.720114,3.458432,0.004248
0,2019-01-01,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,0.116496,0.102682,0.066409,34.409483,1.070490,0.262465,0.092436,0.933164,5.468453,4.151637,0.998070,517.142241,83.019826,1418.196271,0.006531
0,2019-01-01,CAT,124.029999,127.879997,123.000000,118.137177,4783200.0,0.186871,0.107064,0.056932,0.289572,1.428582,0.919490,0.266175,2.135008,2.339630,3.660183,0.803394,4.086316,35.907956,4.375155,0.007280
0,2019-01-01,CSCO,42.279999,43.200001,42.209999,39.496738,23833500.0,0.263373,0.261680,0.098017,0.246218,1.801859,1.677431,1.370671,7.722516,4.244056,7.937160,0.601911,1.512001,28.011871,4.283841,0.008355


In [ ]:
trade.tail()

,date,tic,open,high,low,close,volume,OPM,NPM,ROA,ROE,cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,PE,PB,Div_yield
730,2020-12-31,V,218.399994,219.820007,216.199997,218.074188,5922200.0,0.638994,0.480876,0.094422,0.201545,2.121065,1.969814,1.700081,0.000000,4.725314,2.347866,0.531507,1.134505,153.573372,13.708217,0.001467
730,2020-12-31,VZ,58.060001,58.799999,58.020000,56.870125,12906300.0,0.230962,0.141165,0.041750,0.190741,1.376551,1.210691,0.589158,20.255568,3.797160,5.456577,0.781118,3.568671,13.225610,3.468883,0.011034
730,2020-12-31,WBA,39.330002,40.000000,39.029999,39.166107,7696000.0,0.055546,0.032500,0.058790,0.000000,0.883715,0.201841,0.131341,6.289502,75.704738,7.523900,0.587123,0.000000,11.519443,2.354706,0.006064
730,2020-12-31,WMT,144.199997,144.270004,142.850006,142.989090,5938000.0,0.153981,0.107187,0.154298,0.308230,1.309322,0.971255,0.507827,19.541401,13.960422,6.460123,0.499404,0.997621,155.422924,6.520801,0.002797
730,2020-12-31,XOM,41.470001,41.680000,40.970001,39.949696,22786500.0,-0.016389,-0.017842,-0.007122,-0.014440,0.796498,0.442578,0.077427,5.927586,6.454303,4.660480,0.506747,1.027356,-7.609466,1.076087,0.021777


In [ ]:
ratio_list = ['OPM', 'NPM','ROA', 'ROE', 'cur_ratio', 'quick_ratio', 'cash_ratio', 'inv_turnover','acc_rec_turnover', 'acc_pay_turnover', 'debt_ratio', 'debt_to_equity',
       'PE', 'PB', 'Div_yield']

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2 * stock_dimension + len(ratio_list)* stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 30, State Space: 511


In [ ]:
env_kwargs = {
    "hmax": 100,                    # số cổ phiếu tối đa được mua trên một chứng khoán 
    "initial_amount": 1000000,      # lượng tiền có sẵn ban đầu 
    "buy_cost_pct": 0.001,          # chi phí mua
    "sell_cost_pct": 0.001,         # chi phí bán 
    "state_space": state_space,     # không gian trạng thái 
    "stock_dim": stock_dimension,   # số chiều chứng khoán 
    "tech_indicator_list": ratio_list,  # danh sách các tỷ lệ (chỉ dẫn kỹ thuật)
    "action_space": stock_dimension,  # không gian hành động 
    "reward_scaling": 1e-4        # điểm thưởng
    
}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
# môi trường huấn luyện 
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Mô Hình DDPG Cho Giao Dịch Chứng Khoán Tự Động 

In [ ]:
 agent = DRLAgent(env=env_train)

In [ ]:
agent = DRLAgent(env=env_train)
model_ddpg = agent.get_model("ddpg")

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [ ]:
trained_ddpg = agent.train_model(model= model_ddpg, tb_log_name='ddpg',total_timesteps=50000)

# Thực Hiện Giao Dịch 
Không gian hành động: Mô tả những hành động mà tác nhân tương tác với môi trường được cho phép. Thông thường , action (a) nhận 3 giá trị {-1, 0, 1} tương đương với các hành động { bán, giữ, mua } một cổ phiếu. Bên cạnh đó, một hành động cũng có thể được thực hiện trên nhiều cổ phiếu. 

Chúng ta sử dụng một không gian trạng thái {-k,…,-1, 0, 1, …, k} trong đó k đại diện cho số cổ phiếu. Vd: Mua 100 cổ phiếu là -100, bán 44 cổ phiếu là 44. Không gian hành động liên tục cần được chuẩn hóa thành [-1, 1], bởi vì chiến lược được xác định trên một phân phối Gaussian.

In [ ]:
trade = data_split(processed_full, '2009-01-01','2021-01-01')
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs) 

In [ ]:
trade.head()

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model= trained_ddpg, environment=e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
# Thực hiện hành động trên các loại cổ phiếu 
df_actions.head()

In [ ]:
df_actions.tail()

# Backtest Chiến Lược 


In [ ]:
#  Kết quả của Backtest
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
# Số liệu thống kê cơ bản 
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^GSPC", 
        start = '2019-01-01',
        end = '2021-01-01')

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
# So sánh với sàn S&P 500
print("==============Compare to S&P 500 ===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
# DJIA : ^DJI
backtest_plot(df_account_value, 
             baseline_ticker = '^GSPC', 
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01')